## Configuration

In [ ]:
CONFIG = {
    "data_path": "data\PSD_DE\imaging", # or "data/PSD_DE/watching"
    "label_path": "data/metadata/GT_label.npy",
    "save_path": "checkpoints/eeg2label",

    "seed": 42, 
    "train_valid": [0.8, 0.2], 
    "batch_size": 32,
    "num_workers": 0,

    "input_dim": 62*5,
    "emb_dim": 64,
    "out_dim": 50,

    "learning_rate": 0.001, 
    "valid_steps": 10,
    "epochs": 100, 
}

In [2]:
import torch
import numpy as np
import random
import os

def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

## Load Data

In [3]:
from torch.utils.data import Dataset

from einops import rearrange, repeat

class myDataset(Dataset): 
    '''Generate dataset'''
    def __init__(self, data_path, label_path):
        self.data, self.label = self.data_label(data_path, label_path)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx].astype(np.float32), self.label[idx].astype(np.int64)-1

    def data_label(self, data_path, label_path):
        '''concat the watching or imaging data and label into structured data
        Args:
            data_path: the path of watching or imaging data, corresponding data's shape (2, 5, 50, 62, 5)
                2: PSD and DE, 5: 5 videos, 50: 50 clips per video, 62: 62 electrodes, 5: 5 frequency bands
            label_path: the path of label, corresponding label's shape (5, 50)
                5: 5 videos, 50: 50 clips per video
        Returns:
            structured_data: the structured data, (60*2*5*50, )
                60: 60 experiments, 2: PSD and DE, 5: 5 videos, 50: 50 clips per video
                each elements: 
                    'features':(62*5, ), 62 electrodes, 5 frequency bands
                    'label':int"
        '''
        data = []
        for file in os.listdir(data_path):
            file_path = os.path.join(data_path, file)
            data.append(np.load(file_path))
        data = np.stack(data, axis=0)
        label = np.load(label_path)
        rearrange_data = rearrange(data, 'a b c d e f -> (a b c d) (e f)') # a is number of experiments
        rearrange_label = repeat(label, 'a b -> (repeat a b)', repeat=data.shape[0]*data.shape[1])

        return rearrange_data, rearrange_label

## Load Data

In [ ]:
from torch.utils.data import DataLoader, random_split

def get_dataloader(data_path, label_path, batch_size, num_workers=0):
    '''Generate dataloader'''
    dataset = myDataset(data_path, label_path)
    trainset, validset = random_split(dataset, CONFIG["train_valid"])

    train_loader = DataLoader(
        trainset, 
        batch_size=batch_size,
        shuffle=True,
        drop_last=False,
        num_workers=num_workers, 
        pin_memory=torch.cuda.is_available()
    )
    
    valid_loader = DataLoader(
        validset, 
        batch_size=batch_size,
        shuffle=True,
        drop_last=False,
        num_workers=num_workers, 
        pin_memory=torch.cuda.is_available()
    )

    return train_loader, valid_loader

## Define Model

In [5]:
from models.eeg2label import glfnet

## Loss function

In [6]:
def loss_acc(batch, model, criterion, device):
    '''Forward a batch through the model'''
    input, labels = batch
    input = input.to(device)
    labels = labels.to(device)

    output = model(input)

    loss = criterion(output, labels)

    preds = output.argmax(1)
    accuracy = torch.mean((preds == labels).float())

    return loss, accuracy

## Valid function

In [7]:
from tqdm import tqdm

def valid(data_loader, model, criterion, device): 
    '''Validate the model on the validation set'''
    model.eval()
    running_loss = 0
    running_accuracy = 0
    pbar = tqdm(total = len(data_loader.dataset), ncols=0, desc='Valid')

    for i, batch in enumerate(data_loader):
        with torch.no_grad():
            loss, accuracy = loss_acc(batch, model, criterion, device)
            running_loss += loss.item()
            running_accuracy += accuracy.item()
        pbar.update(data_loader.batch_size)
        pbar.set_postfix(
			loss=f"{running_loss / (i+1):.2f}",
			accuracy=f"{running_accuracy / (i+1):.2f}",
		)
    pbar.close()
    model.train()

    return running_accuracy / len(data_loader)

# Main Function

In [8]:
from torch import nn

def main():
    set_seed(CONFIG["seed"])

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"[Info]: Use {device} now!")

    train_loader, valid_loader = get_dataloader(CONFIG["data_path"], CONFIG["label_path"], CONFIG["batch_size"], num_workers=CONFIG["num_workers"])
    print(f"[Info]: Finish loading data!")

    model = glfnet(input_dim=CONFIG["input_dim"], emb_dim=CONFIG["emb_dim"], out_dim=CONFIG["out_dim"]).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=CONFIG["learning_rate"])
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200 * len(train_loader))

    best_accuracy = 0
    best_state_dict = None

    pbar = tqdm(range(CONFIG['epochs']), desc='Train', unit='epoch', dynamic_ncols=True)
    for epoch in pbar:
        running_loss = 0.0
        running_accuracy = 0.0
        for i, batch in enumerate(train_loader):
            loss, accuracy = loss_acc(batch, model, criterion, device)
            batch_loss = loss.item()
            batch_accuracy = accuracy.item()

            loss.backward()
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

            running_loss += batch_loss
            running_accuracy += batch_accuracy

        train_loss = running_loss/len(train_loader)
        train_acc = running_accuracy/len(train_loader)
        pbar.set_postfix(train_loss=train_loss, train_acc=train_acc)

        if (epoch + 1) % CONFIG['valid_steps'] == 0:
            valid_acc = valid(valid_loader, model, criterion, device)
            pbar.write(f"[Info]: Valid acc: {valid_acc:.4f}")
            if valid_acc > best_accuracy:
                best_accuracy = valid_acc
                best_state_dict = model.state_dict()
                pbar.write(f"[Info]: 😄 Best acc updated: {best_accuracy:.4f}")

    pbar.close()

    torch.save(best_state_dict, os.path.join(CONFIG["save_path"], "best_model.pth"))
    print("="*50, f"\n[Info]: Best model saved to {os.path.join(CONFIG['save_path'], 'best_model.pth')}")

    

## Start training!

In [9]:
if __name__ == '__main__':
    main()

[Info]: Use cpu now!
[Info]: Finish loading data!


Train:   0%|          | 0/100 [00:00<?, ?epoch/s]d:\app\miniconda\envs\eeg_test\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Valid: 608it [00:00, 6105.99it/s, accuracy=0.04, loss=3.01]rain_acc=0.055, train_loss=3]    
Train:  10%|█         | 10/100 [00:04<00:48,  1.87epoch/s, train_acc=0.055, train_loss=3]

[Info]: Valid acc: 0.0428
[Info]: 😄 Best acc updated: 0.0428


Train:  12%|█▏        | 12/100 [00:06<00:44,  1.99epoch/s, train_acc=0.0542, train_loss=3]


KeyboardInterrupt: 